In [1]:
"""
Script extracting wavenumbers, temperature, hot/cold status from temperature '.log' and raw spectra '.txt' files
"""

import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import math as m
import pandas as pd
import glob
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import itertools

In [2]:
# VARIABLES
file_name = 'IC_raw_spectra_(2019_02_25_'

"""Sand"""
path_raw = '//icnas4.cc.ic.ac.uk/rs5115/MSci Project/sand_outside' #path containing all raw spectra

In [3]:
#FUNCTIONS

def read_file(directory, filename, separator, rowstoskip, head='infer'):
    """
    Reads an external file and outputs it as a Pandas Dataframe
    Input: Directory, Filename (Strings)
    Output: Pandas Dataframe
    """
    path=os.path.join(directory, filename)
    dataset = pd.read_csv(path, header = head, sep=separator, skiprows=rowstoskip)
    
    return dataset


def planck(wavenumber, temperature): # T in Kelvin, wavenumber in cm^-1
    """
    Calculates spectral radiance of a black body at some wavenumber and temperature
    Input: wavenumber (units: cm^{-1}), and temperature T (units: Kelvin) - (Integers or Floats)
    Output: spectral radiance (units: W m^{-2} sr^{-1} cm^{-1}) - (Float)
    """
    h = 6.626e-34
    c = 3.0e+8
    k = 1.381e-23

    planck_a = 2*h*(c**2)
    planck_e = h*c/k
    radiance = 100*planck_a*((wavenumber*100)**3)/(np.exp(planck_e*(wavenumber*100)/temperature)-1)

    return radiance


def read_raw_spectra(directory, filename):
    """
    Reads spectrum file and calculates the average of the 4 spectra
    Input: Directory, Filename (Strings)
    Output: average_spectrum (NumPy Array)
    """
    file_data = read_file(path_raw, filename, separator='  ', rowstoskip=5, head=None)
    spectrum1 = file_data[1].values
    spectrum2 = file_data[2].values
    spectrum3 = file_data[3].values
    spectrum4 = file_data[4].values
    
    return spectrum1,spectrum2,spectrum3,spectrum4

In [4]:
# CODE

#create list of file paths starting with file_name from 'path_raw' directory
measurements = len(glob.glob(path_raw + '/'+ file_name + '*')) #filepaths of all 'IC_raw_spectra' files
m_numbers = list(range(measurements)) #list of file name endings (integers)

for i in m_numbers:
    #checks that files have associated temperature measurements, and removes the files if not
    temp_check = read_file(path_raw, file_name + '{0:03}'.format(i) + '.txt', separator=',', rowstoskip=None, head=None).iloc[1,0]
    temp_check_float = float(temp_check.split()[0])
    if temp_check_float == 0.0:
        m_numbers.pop(i)

files = [file_name + '{0:03}'.format(num) + '.txt' for num in m_numbers] #final list of strings of file names, in order

AttributeError: module 'pandas' has no attribute 'read_csv'

In [ ]:
columns_ = ['RT0','RT1','RT2','Dummy','RT3','RT4','HBB_temp','CBB_temp','Time', 'Orientation'] #columns for Pandas DataFrame

read_temps = []

for i in np.arange(0,len(files)):
    #Go through files and retrieve temperatures and orientations of spectra
    read_temps.append( read_file(path_raw, files[i], separator=',', rowstoskip=None, head=None).iloc[1:5,0].str.split() )
    read_temps_flat = list(itertools.chain(*read_temps)) #flatten read_temps_i

times_temps = pd.DataFrame(read_temps_flat, columns = columns_)

In [ ]:
times_temps.drop(['Dummy'], axis = 1, inplace = True, errors = 'ignore')
times_temps['hc'] = np.nan #add empty 'hc' column

times_temps.loc[times_temps['Orientation'] == '270', 'hc'] = 'h'             #If 'Orientation'=270 degrees from nadir, then facing HBB
times_temps.loc[times_temps['Orientation'] == '225', 'hc'] = 'c'             #facing CBB
times_temps.loc[times_temps['Orientation'] == '20', 'hc'] = 'sand'           #sand view at 20 degrees from nadir
times_temps.loc[times_temps['Orientation'] == '160', 'hc'] = 'sky_sand'      #sky view for sand measurement
times_temps.loc[times_temps['Orientation'] == '55', 'hc'] = 'water'          #water view at 55 degrees from nadir
times_temps.loc[times_temps['Orientation'] == '125', 'hc'] = 'sky_water'     #sky view for water measurement

columns_.pop(columns_.index('Time')) #Remove b from list
columns_.pop(columns_.index('Orientation'))
times_temps = times_temps[['Time']+columns_] #Create new dataframe with columns in the order you want

In [ ]:
times_temps